In [2]:
import time
start = time.time()
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import glob
print(time.time()-start,'s')

0.5693860054016113 s


### Algorithm 1

Just put it to another file because it was consuming space here.

### Algorithm 2
Better approach

In [110]:
def order_points(pts):

    rect = np.zeros((4, 2), dtype = "float32")
 

    s = pts.sum(axis = 1)
#     print(s)
#     print(pts)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
 

    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
 
    return rect

In [ ]:
def four_point_transform(image, pts):

    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([
              [0, 0],
              [maxWidth - 1, 0],
              [maxWidth - 1, maxHeight - 1],
              [0, maxHeight - 1]], dtype = "float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
 
    return warped

In [ ]:
def cont(img, gray, user_thresh, crop, filename):
    found = False
    cwd = os.getcwd() + '/crop/'
    orig_thresh = user_thresh
    im_h, im_w = img.shape[:2]
    while found == False: # repeat to find the right threshold value for finding a rectangle
        if user_thresh < 200:
            user_thresh = orig_thresh + 5
            orig_thresh = user_thresh 
        print(user_thresh)
        ret,thresh = cv2.threshold(gray,user_thresh,255,cv2.THRESH_BINARY)
        _,contours,hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        im_area = im_w * im_h
        for cnt in contours:         
            area = cv2.contourArea(cnt)
            if area > (im_area/6) and area < (im_area/1.01):
                epsilon = 0.1*cv2.arcLength(cnt,True)
                approx = cv2.approxPolyDP(cnt,epsilon,True)
                if len(approx) == 4:
                    found = True
                else:
                    user_thresh = user_thresh - 1
                    break                
                rect = np.zeros((4, 2), dtype = "float32")
                rect[0] = approx[0]
                rect[1] = approx[1]
                rect[2] = approx[2]
                rect[3] = approx[3]
                
                dst = four_point_transform(img, rect)
                dst_h, dst_w = dst.shape[:2]
                img = dst[crop:dst_h-crop, crop:dst_w-crop]
                dst_h, dst_w = img.shape[:2]
                #print("Saving to "+cwd+"crop_"+filename)
                #cv2.imwrite(cwd+"crop_"+filename, img, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
                #res = cv2.resize(img,(dst_w/6, dst_h/6), interpolation = cv2.INTER_CUBIC)
          
    return found, im_w, im_h, img

In [ ]:
def main(thresh, crop, filename):
    img = cv2.imread(filename)
    print("Opening: "+filename)

    #add white background (in case one side is cropped right already, otherwise script would fail finding contours)
    #img = cv2.copyMakeBorder(img,100,100,100,100, cv2.BORDER_CONSTANT,value=[255,255,255])
    im_h, im_w = img.shape[:2]
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    res_gray = cv2.resize(img,(int(im_w/6), int(im_h/6)), interpolation = cv2.INTER_CUBIC)
    found, im_w, im_h, image = cont(img, gray, thresh, crop, filename)
    return image

In [ ]:
os.system("mkdir crop")
thresh = 166
crop = 15
files = []
types = ('*.bmp','*.BMP','*.tiff','*.TIFF','*.tif','*.TIF','*.jpg', '*.JPG','*.JPEG', '*.jpeg', '*.png', '*.PNG', ) #all should work but only .jpg was tested
for t in types:
    if glob.glob(t) != []:
        files.append(glob.glob(t))
# for f in files[0]:
img1 = main(thresh, crop, 'form-samples/filled-s1.jpg')
print(img1.shape)

## Extraction

### Algorithm-1
If the form is standard and perfect this code will work better. Can generate both grayscale and colour images.

In [22]:
image = cv2.imread('form-samples/filled-s1.jpg')
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

In [23]:
height, width = gray.shape
print(height)
print(width)

#for Residential Address
out = cv2.rectangle(gray,(int(width/10),int(17*height/23)),(int(10*width/11),int(9*height/10)),(0,255,0),2)
## out = cv2.rectangle(gray,(int(width/10),int(18*height/25)),(int(15*width/16),int(13*height/14)),(0,255,0),2)
# Branch 
out = cv2.rectangle(gray,(int(width/10),int(3*height/17)),(int(13*width/21),int(6*height/32)),(0,255,0),2)
out = cv2.rectangle(gray, (int(0),int(18*height/25)),(width,int(25*height/26)), (0), thickness = -1)


# Residential address
out1 = gray[int(17*height/23):int(9*height/10),int(width/10):int(10*width/11)]

#Branch
out1 = gray[int(3*height/17):int(6*height/32),int(width/10):int(13*width/21)]
print(out1.shape)

cv2.imwrite('Out.jpg',out)

3507
2481
(39, 1287)


True

### Algorithm 2
Even if the form is skewed but has darker background the algorithm takes care of it and crops the address field

In [123]:
img1 = cv2.imread('form-samples/filled-s1.jpg')
crop = cv2.imread('Residential_address.jpg')
gray_main = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
gray_crop = cv2.cvtColor(crop,cv2.COLOR_BGR2GRAY)
height_main, width_main = gray_main.shape
height_crop, width_crop = gray_crop.shape
height = height_main
width = width_main
print(height_main, width_main)
print(height_crop, width_crop)
#cv2.imwrite('Out.jpg',gray_main)

3507 2481
285 1090


#### Approach 1

In [102]:
#Trying feature matching using ORB
orb = cv2.ORB_create()
mask = np.zeros(gray_main.shape[:2], dtype=np.uint8)
#cv2.rectangle(mask, (int(width/10),int(18*height/25)),(int(15*width/16),int(13*height/14)), (255), thickness = -1)
#cv2.rectangle(mask,(int(width/10),int(2*height/17)),(int(13*width/21),int(4*height/29)),(255),-1)
cv2.rectangle(mask, (int(width/10),int(18*height/25)),(int(15*width/16),int(13*height/14)), (255), thickness = -1)
# find the keypoints and descriptors with SIFT
kp1, des1 = orb.detectAndCompute(gray_main,mask)
kp2, des2 = orb.detectAndCompute(gray_crop,None)


In [103]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

matches = bf.match(des1,des2)

matches = sorted(matches, key = lambda x:x.distance)

img = cv2.drawMatches(gray_main,kp1,gray_crop,kp2,matches,None, flags=2)

cv2.imwrite('Out1.jpg',img)

True

#### Approach 2

In [124]:
#Trying feature matching using SIFT
# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()

mask = np.zeros(gray_main.shape[:2], dtype=np.uint8)
#mask for residential address
cv2.rectangle(mask, (int(0),int(18*height/25)),(width,int(25*height/26)), (255), thickness = -1)

#mask for branch
#cv2.rectangle(mask,(int(width/10),int(3*height/27)),(int(10*width/11),int(3*height/20)),(255),-1)

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(gray_main,mask)
kp2, des2 = sift.detectAndCompute(gray_crop,None)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(des1,des2,k=2,mask=mask)

# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.8*n.distance:
        good.append(m)

In [125]:
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
matchesMask = mask.ravel().tolist()

h,w = gray_main.shape
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
dst = cv2.perspectiveTransform(pts,M)


img2 = cv2.polylines(gray_crop,[np.int32(dst)],True,255,3, cv2.LINE_AA)

In [126]:
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

img3 = cv2.drawMatches(gray_main,kp1,gray_crop,kp2,good,None,**draw_params)
cv2.imwrite('Out1.jpg',img3)

True

In [127]:
#For approach 1
# src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ])
# point = src_pts


# #For approach 2
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ])
point = src_pts

# #for Residential address
point = src_pts
points = point[point[:,1]>(7*height)/10]

#for Branch name
# point = src_pts
# points = point[point[:,1]<(height)/8]

#for Name
# point = src_pts
# points = point[point[:,1]>(height)/8]
# points = points[points[:,1]<2*(height)/8]

In [128]:
def get_outer_points(pts,crop):

    rect = np.zeros((4, 2), dtype = "float32")
 
    min_x = min(pts[:,0])
    min_y = min(pts[:,1])
    max_x = max(pts[:,0])
    max_y = max(pts[:,1])
    #s = pts.sum(axis = 1)
    #print(s)
    rect[0][0] = min_x
    rect[0][1] = min_y
    rect[2][0] = max_x
    rect[2][1] = max_y
 

    #diff = np.diff(pts, axis = 1)
    #print(diff)
    rect[1][0] = max_x
    rect[1][1] = min_y
    rect[3][0] = min_x
    rect[3][1] = max_y
    
    #cropping a little more towards left
    rect[0][0] -=crop
    rect[3][0] -=crop
    #cropping a little more towards right
    rect[1][0] +=crop
    rect[2][0] +=crop
    #cropping a little more towards bottom
    rect[3][1] +=crop
    rect[2][1] +=crop
 
    return rect

In [131]:
def four_point_warp(image, pts,crop):

    rect = get_outer_points(pts,crop)
    (tl, tr, br, bl) = rect
    print(rect)

    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([
              [0, 0],
              [maxWidth - 1, 0],
              [maxWidth - 1, maxHeight - 1],
              [0, maxHeight - 1]], dtype = "float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    cv2.imwrite('Out.jpg',warped)
 
    return warped

In [132]:
crop = 10
dst = four_point_warp(gray_main, points,crop)
dst_h, dst_w = dst.shape[:2]
img = dst[crop:dst_h-crop, crop:dst_w-crop]
dst_h, dst_w = img.shape[:2]
#cv2.imwrite('Out1.jpg',img)

[[  241.08084106  2525.06274414]
 [ 2126.73852539  2525.06274414]
 [ 2126.73852539  3254.41186523]
 [  241.08084106  3254.41186523]]


In [83]:
time.time()-start

2659.2945823669434

In [58]:
dst.shape


(963, 2065)

In [ ]:
out1.shape